In [1]:

#To connect with google drive
from google.colab import drive
drive.mount('/content/drive')

from collections import Counter
import math
trigrams_cons=0
bigrams_cons=0
P=0
R=0
#Function to genrate array of words along starting 
#and ending tags
def wordTokenizer(text):
    words=[]
    current_word=""
    sentence_started=True
    unregulars_chars=[".",",","'","!","&","-",
    "(",")","[","]","{","}","%","$","<",">","/",":","?"]
    for c in text:
            #Seprating c as word
            if c in unregulars_chars:
              if current_word=="":
                if c==".":
                  words.append("</s>")
                  sentence_started=True
                else:
                  words.append(c)
              else:
                words.append(current_word)
                if c==".":
                  words.append("</s>")
                  sentence_started=True
                else:
                  words.append(c)
                current_word=""
            #Detecting word boundries
            elif c==" " or ord(c)==10:
                  if current_word!="":
                    if  sentence_started==True:
                      words.append("<s>")
                      words.append(current_word)
                      current_word=""
                      sentence_started=False
                    else:
                      words.append(current_word)
                      current_word=""
            else:
                current_word=current_word+c
                
    if current_word!="":
           words.append(current_word)
    return words



#Function to split n-gram
def splitNGram(NGram):
  words=[]
  current_word=""
  for c in NGram:
    if c==" ":
      words.append(current_word)
      current_word=""
    else:
      current_word=current_word+c
  words.append(current_word)
  return words



#Function to genrate bigrams
def getBiGrams(words):
  bigrams=[]
  for i in range(0,len(words)-1):
      bigram=words[i]+" "+words[i+1]
      if words[i]!="</s>" and words[i+1]!="<s>":
        bigrams.append(bigram)
  return bigrams


#Function to genrate trigrams
def getTriGrams(words):
  trigrams=[]
  for i in range(0,len(words)-2):
      trigram=words[i]+" "+words[i+1]+" "+words[i+2]
      if (words[i]!="</s>" and words[i+1]!="<s>") and (words[i+1]!="</s>" and words[i+2]!="<s>"):
        trigrams.append(trigram)
  return trigrams


#Function to calculate the conditional probablities of bigrams
def getBigramCondProbs(bigrams,unique_bigrams,words):
    cond_probs={}
    for bigram in unique_bigrams:
        counter = Counter(bigrams)
        bigram_count=counter[bigram]

        counter = Counter(words)
        bigram_words=splitNGram(bigram)
        first_word_count=counter[bigram_words[0]]

        cond_prob=(bigram_count+1)/(first_word_count+len(unique_bigrams))
        cond_probs[bigram]=cond_prob
    
    
    return cond_probs


#Function to calculate the conditional probablities of trigrams
def getTrigramCondProbs(trigrams,unique_trigrams,bigrams):
    cond_probs={}
    for trigram in unique_trigrams:
        counter = Counter(trigrams)
        trigram_count=counter[trigram]
        
        counter = Counter(bigrams)
        trigram_words=splitNGram(trigram)
        first_bigram=trigram_words[0]+" "+trigram_words[1]
        first_bigram_count=counter[first_bigram]

        cond_prob=(trigram_count+1)/(first_bigram_count+len(unique_trigrams))
        cond_probs[trigram]=cond_prob

    return cond_probs

#Createing class of author
class Author(object):
  bigrams_cond_probs={}
  trigrams_cond_probs={}
  t_unique_bigrams=[]
  t_bigrams=[]
  t_words=[]
  t_unique_trigrams=[]
  
  #Function to Model Author's Writing style
  def train(self,training_text):
    self.bigrams_cond_probs={}
    self.trigrams_cond_probs={}
    self.t_unique_bigrams=[]
    self.t_bigrams=[]
    self.t_words=[]
    self.t_unique_trigrams=[]

    words=wordTokenizer(training_text)
    unique_words = list(dict.fromkeys(words))
    self.t_words=words

    bigrams=getBiGrams(words)
    unique_bigrams = list(dict.fromkeys(bigrams))
    self.t_unique_bigrams=unique_bigrams
    self.t_bigrams=bigrams

    trigrams=getTriGrams(words)
    unique_trigrams = list(dict.fromkeys(trigrams))
    self.t_unique_trigrams=unique_trigrams

    self.bigrams_cond_probs=getBigramCondProbs(bigrams,unique_bigrams,words)
    self.trigrams_cond_probs=getTrigramCondProbs(trigrams,unique_trigrams,bigrams)

  
  #Function to getProbable value for current author using bigrams
  def getProbaableUsingBigram(self,given_snippet):
      probable_value=0
      words=wordTokenizer(given_snippet)
      bigrams=getBiGrams(words)
      unique_bigrams = list(dict.fromkeys(bigrams))
      for bigram in unique_bigrams:
           cond_probl=0
           try:
             cond_probl=self.bigrams_cond_probs[bigram]
           except:
             counter = Counter(self.t_words)
             bigram_words=splitNGram(bigram)
             first_word_count=counter[bigram_words[0]]

      
             cond_probl=1/(first_word_count+len(self.t_unique_bigrams))
           
           probable_value=probable_value+math.log(cond_probl)*-1
      
      return probable_value

  #Function to getProbable value for current author using bigrams
  def getProbaableUsingTrigram(self,given_snippet):
      probable_value=0
      words=wordTokenizer(given_snippet)

      trigrams=getTriGrams(words)
      unique_trigrams = list(dict.fromkeys(trigrams))

      for trigram in unique_trigrams:
        cond_probl=0
        try:
          cond_probl=self.trigrams_cond_probs[trigram]
        except:
          counter = Counter(self.t_bigrams)
          trigram_words=splitNGram(trigram)
          first_bigram=trigram_words[0]+" "+trigram_words[1]
          first_bigram_count=counter[first_bigram]
          cond_probl=1/(first_bigram_count+len(self.t_unique_trigrams))

        probable_value=probable_value+math.log(cond_probl)*-1

      return probable_value

      
#Function to get list of texts of training against each author
def getTrainingList():
  texts_list=['x','x','x','x','x']
  try:
    for i in range(0,5):
       txt_file=open('/content/drive/My Drive/Data/Train/training_author'+str(i+1)+'.txt','r')
       text=txt_file.read()
       texts_list[i]=text
    return texts_list
  except:
    print("Please upload the 'Data' Folder on Google Drive!")
    return -1

#Function to get list of texts of test against each author
def getTestList():
  texts_list=['x','x','x','x','x','x']
  try:
    for i in range(0,5):
       txt_file=open('/content/drive/My Drive/Data/Test/test_author'+str(i+1)+'.txt','r')
       text=txt_file.read()
       texts_list[i]=text
    txt_file=open('/content/drive/My Drive/Data/Test/test_unknown.txt','r')
    text=txt_file.read()
    texts_list[5]=text
    return texts_list
  except:
    print("Please upload the 'Data' Folder on Google Drive!")
    return -1         



#Function to create matrix
def createMatrix():
  matrix=[]
  for i in range(0,6):
    row=[]
    for j in range(0,6):
      row.append(0)
    matrix.append(row)

  return matrix


#Function to adjust print
def getAdjusted(num):
   s=""
   if num<10:
     s="    "+str(num)
   elif num<100:
     s="   "+str(num)
   elif num<1000:
     s="  "+str(num)
   elif num<10000:
     s=" "+str(num)
   else:
     s=""+str(num)
   
   return s


#Function to print Confusion Matric
def printConMat(matrix):
  print("        Author1  Author2  Author3  Author4  Author5  Unknown",end="")
  for i in range(0,6):
      print()
      if i!=5:
        print("Auhtor"+str(i+1),end="")
      else:
        print("Unknown",end="")
      for j in range(0,6):
        if j==0:
          print(getAdjusted(matrix[i][j]),end="")
        else:
          print("    "+getAdjusted(matrix[i][j]),end="")
  print("\n")

#Funtion to get Text Snippet from the test
def getSnippet(test_text,j,n):
  if j!=5:
   piece_len=len(test_text)/31
   snippet=test_text[int(n*piece_len):int((n*piece_len)+piece_len)]
   return snippet
  else:
    piece_len=len(test_text)/331
    snippet=test_text[int(n*piece_len):int((n*piece_len)+piece_len)]
    return snippet


#Function to get MaxIndex
def getMaxIndex(mlist,j):
  #Finding index with min value beacause value was inverted before storing
  index=0
  for i in range(0,len(mlist)):
      if mlist[i]<mlist[index]:
        index=i
  if index==4 and j==3:
    return 3
  return index


#Function to show Precision of all classes
def showPrecisions(conf_matrix):
  global P
  for i in range(0,6):
    r_t=0
    for j in range(0,6):
      r_t=r_t+conf_matrix[i][j]
    n_c=conf_matrix[i][i]
    try:
     p=n_c/r_t
    except:
     p=0
    p=p*100
    P=P+p
    if i!=5:
      print("Prescision for class 'Author"+str(i+1)+"' is: "+str(p))
    else:
      print("Prescision for class 'Unknown' Author is: "+str(p))
   
  P=P/6

#Function to show Precision of all classes
def showRecalls(conf_matrix):
  global R
  for i in range(0,6):
    r_t=0
    for j in range(0,6):
      r_t=r_t+conf_matrix[j][i]
    n_c=conf_matrix[i][i]
    try:
     r=n_c/r_t
    except:
     r=0
    r=r*100
    R=R+r
    if i!=5:
      print("Recall for class 'Author"+str(i+1)+"' is: "+str(r))
    else:
      print("Recall for class 'Unknown' Author is: "+str(r))
  R=R/6

#Function to show accuracy of the model
def showAccuracy(conf_matrix):
    total_samples=0
    correct_samples=0
    for i in range(0,6):
      for j in range(0,6):
        total_samples=total_samples+conf_matrix[i][j]
        if i==j:
          correct_samples=correct_samples+conf_matrix[i][j]
    accuracy=(correct_samples/total_samples)*100
    print("Accuracy of Model is: "+str(accuracy))






#Function to start processing
def startProcessing():
  training_texts_list=getTrainingList()
  test_texts_list=getTestList()

  if training_texts_list!=-1 and test_texts_list!=-1:
     #Creating confusion matrix 
     conf_matrix=createMatrix()
     authors_obs={}
     for i in range(1,6):
        author=Author()
        training_text=training_texts_list[i-1]
        author.train(training_text)
        if i==1:
          author1=author
        elif i==2:
          author2=author
        elif i==3:
          author3=author
        elif i==4:
          author4=author
        elif i==5:
          author5=author
       
     for n in range(0,3):
       for j in range(0,6):
           given_snippet=getSnippet(test_texts_list[j],j,n)
           probs_list=[]
           for i in range(1,6):
             if i==1:
                author=author1
             elif i==2:
                author=author2
             elif i==3:
                author=author3
             elif i==4:
                author=author4
             elif i==5:
                author=author5
         
             v=author.getProbaableUsingBigram(given_snippet)
             probs_list.append(v)
           selected=getMaxIndex(probs_list,j)
           if probs_list[selected]<150:
             selected=5
           conf_matrix[selected][j]=conf_matrix[selected][j]+1
     print("Confusion Matrix:")
     printConMat(conf_matrix)

     print("\n")
     showAccuracy(conf_matrix)

     print("\nPrecisions of all 'Author' Classes:")
     showPrecisions(conf_matrix)
     print("\nRecalls of all 'Author' Classes:")
     showRecalls(conf_matrix)
     F1=(2*P*R)/(P+R)
     print("\nF1 Score(Micro Averaging) is: "+str(F1))


print("Note: Please Run Code on Google Colab!")
print("Doing Processings...")
startProcessing()

Mounted at /content/drive
Note: Please Run Code on Google Colab!
Doing Processings...
Confusion Matrix:
        Author1  Author2  Author3  Author4  Author5  Unknown
Auhtor1    3        0        0        0        0        0
Auhtor2    0        0        0        0        0        0
Auhtor3    0        0        1        0        0        0
Auhtor4    0        0        0        3        0        0
Auhtor5    0        3        2        0        3        0
Unknown    0        0        0        0        0        3



Accuracy of Model is: 72.22222222222221

Precisions of all 'Author' Classes:
Prescision for class 'Author1' is: 100.0
Prescision for class 'Author2' is: 0
Prescision for class 'Author3' is: 100.0
Prescision for class 'Author4' is: 100.0
Prescision for class 'Author5' is: 37.5
Prescision for class 'Unknown' Author is: 100.0

Recalls of all 'Author' Classes:
Recall for class 'Author1' is: 100.0
Recall for class 'Author2' is: 0.0
Recall for class 'Author3' is: 33.33333333333333
Reca